# US Stock Screening (Post Market) v0

In this notebook we try to achieve
- Screen US Common Stock with previous day Volume
- Send Telegram notice

Timeline as follow:

|| T0 | T1 | T2 | T3 |
| --- | --- | --- | --- | --- |
| Expected trend | x | Volume Rise | Volume Drop | Price Rise |
| Action | x | **Create Potential Stocks List** | Enter Market near day end | |

Import Library

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

Import Token

In [2]:
from secrets import IEX_SANDBOX_API_TOKEN as IEX_CLOUD_API_TOKEN
baseUrl = f'https://sandbox.iexapis.com/stable'

# from secrets import IEX_CLOUD_API_TOKEN
# baseUrl = f'https://cloud.iexapis.com/stable'

IEX Cloud API test call

In [3]:
symbol = 'AMZN'
symbolString = 'AAPL,TSLA,BIAF,AMZN'

# quote of a stock
batchQuotePreviousUrl = f'/stock/market/batch?symbols={symbolString}&types=quote,previous&token={IEX_CLOUD_API_TOKEN}'

apiUrl = baseUrl + batchQuotePreviousUrl
# data = requests.get(apiUrl)
# print(data.status_code)
# print(data.text)
# data = requests.get(apiUrl).json()
# data

## Preparation

Fetch all available symbol, filter with "US" and "common stock".

In [4]:
refDataUrl = f'/ref-data/symbols?token={IEX_CLOUD_API_TOKEN}'
apiUrl = baseUrl + refDataUrl

data = requests.get(apiUrl).json()
# data

In [5]:
allSymbolsColumns = [
    "symbol",
    "name",
    "date",
    "type",
    "iexId",
    "region",
    "currency",
    "isEnabled",
    "figi",
    "cik",
]

allSymbolsDataFrame = pd.DataFrame(columns = allSymbolsColumns)

for symbol in data:
    if symbol['type'] == 'cs' and symbol['region'] == 'US':
        allSymbolsDataFrame = allSymbolsDataFrame.append(
            pd.Series(
                [
                    symbol['symbol'],
                    symbol['name'],
                    symbol['date'],
                    symbol['type'],
                    symbol['iexId'],
                    symbol['region'],
                    symbol['currency'],
                    symbol['isEnabled'],
                    symbol['figi'],
                    symbol['cik'],
                ],
                index = allSymbolsColumns
            ),
            ignore_index = True
        )
allSymbolsDataFrame

,symbol,name,date,type,iexId,region,currency,isEnabled,figi,cik
0,A,tocc.o eg enneAnilhlTiIgs,2022-07-12,cs,IEX_46574843354B2D52,US,USD,True,BBGV326000DC,1145365
1,AA,loo ACarpc,2022-07-12,cs,IEX_4238333734532D52,US,USD,True,3B3BDB3G0HT0,1682266
2,AAC,slpraaoC in- As eo trii AoucsoArsnqiCt,2022-07-12,cs,None,US,USD,True,920ZCZY1B0BG,1858074
3,AACI,niod icpaCamrsotiAurI Aq,2022-07-12,cs,None,US,USD,True,10BR36BGX107,1872348
4,AADI,Acicsdna i cienIBeo,2022-07-12,cs,None,US,USD,True,G02B7NB2DT0W,1486457
...,...,...,...,...,...,...,...,...,...,...
6209,ZWS,eatE SunpouokZtlsCri nlaWo yrr,2022-07-12,cs,None,US,USD,True,8B0RB008GHN0,1474044
6210,ZY,IeZnmnceygr,2022-07-12,cs,None,US,USD,True,0H7P470N7BBG,1713897
6211,ZYME,ecsmko yZwIrn,2022-07-12,cs,IEX_5253315838322D52,US,USD,True,00DGBBG55JL1,1442638
6212,ZYNE,nunPbr aZarieecyIsahtmccla,2022-07-12,cs,IEX_4E3154424A382D52,US,USD,True,SBG8BB7B070B,1660687


Define chunks to prepare for batch API call

In [6]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
symbolGroups = list(chunks(allSymbolsDataFrame['symbol'], 100))
symbolStrings = []
for i in range(0, len(symbolGroups)):
    symbolStrings.append(','.join(symbolGroups[i]))
#     print(symbol_strings[i])

symbolStrings

['A,AA,AAC,AACI,AADI,AAIC,AAL,AAMC,AAME,AAN,AAOI,AAON,AAP,AAPL,AAQC,AAT,AATC,AAU,AAWW,ABBV,ABC,ABCB,ABCL,ABEO,ABG,ABGI,ABIO,ABM,ABMD,ABNB,ABOS,ABR,ABSI,ABST,ABT,ABTX,ABUS,ABVC,AC,ACA,ACAB,ACAD,ACAH,ACAQ,ACAX,ACB,ACBA,ACC,ACCD,ACCO,ACDI,ACEL,ACER,ACET,ACEV,ACGL,ACHC,ACHR,ACHV,ACI,ACII,ACIU,ACIW,ACLS,ACLX,ACM,ACMR,ACN,ACNB,ACON,ACOR,ACP,ACQR,ACR,ACRE,ACRO,ACRS,ACRX,ACST,ACT,ACTD,ACTG,ACU,ACV,ACVA,ACXP,ADAL,ADBE,ADC,ADCT,ADER,ADES,ADEX,ADGI,ADI,ADIL,ADM,ADMA,ADMP,ADN',
 'ADNT,ADOC,ADP,ADPT,ADRA,ADRT,ADSE,ADSK,ADT,ADTH,ADTN,ADTX,ADUS,ADV,ADVM,ADX,AE,AEAC,AEAE,AEE,AEF,AEHA,AEHL,AEHR,AEI,AEIS,AEL,AEM,AEMD,AEO,AEP,AER,AERC,AERI,AES,AESE,AEVA,AEY,AEYE,AEZS,AFAC,AFAQ,AFAR,AFB,AFBI,AFCG,AFG,AFIB,AFL,AFMD,AFRI,AFRM,AFT,AFTR,AFYA,AG,AGAC,AGBA,AGCB,AGCO,AGD,AGE,AGEN,AGFS,AGFY,AGGR,AGI,AGIL,AGIO,AGL,AGLE,AGM,AGMH,AGNC,AGO,AGR,AGRI,AGRO,AGRX,AGS,AGTC,AGTI,AGX,AGYS,AHCO,AHH,AHPA,AHPI,AHRN,AHT,AI,AIB,AIF,AIG,AIHS,AIKI,AIM,AIMC,AIN,AINC',
 'AINV,AIO,AIP,AIR,AIRC,AIRG,AIRI,AIRS,AIRT,AIT,AIV,AIZ,AJG,AJRD,

## Data Fetching

Create main DataFrame, including volume of today, previous trading date.

In order to get the previous trading date, we need to call **/previous**, or else we can use **/quote/previousVolume** directly

Data imported from IEX Cloud API.

In [7]:
potentialDataFrameColumns = [
    'Ticker',
    'Price',
    '30 day Average Volume',
    'T0 Date',
    'T0 Volume',
    'T1 Volume',
    'T1 Volume Ratio',
]

potentialDataFrame = pd.DataFrame(columns = potentialDataFrameColumns)

for symbolString in symbolStrings:
    batchApiUrlCall = baseUrl + f'/stock/market/batch?symbols={symbolString}&types=quote,previous&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batchApiUrlCall).json()
#     print(data)
#     print(data.status_code)
    for symbol in symbolString.split(','):
        try:
            latestPrice = data[symbol]['quote']['latestPrice']
        except KeyError:
            latestPrice = np.NaN
        try:
            avgTotalVolume = data[symbol]['quote']['avgTotalVolume']
        except KeyError:
            avgTotalVolume = np.NaN
        try:
            latestVolume = data[symbol]['quote']['latestVolume']
        except KeyError:
            latestVolume = np.NaN
            
        try:
            t0Date = data[symbol]['previous']['date']
        except (KeyError, TypeError):
            t0Date = np.NaN
        try:
            t0Volume = data[symbol]['previous']['volume']
        except (KeyError, TypeError):
            t0Volume = np.NaN
            
        potentialDataFrame = potentialDataFrame.append(
            pd.Series(
                [
                    symbol,
                    latestPrice,
                    avgTotalVolume,
                    t0Date,
                    t0Volume,
                    latestVolume,
                    'N/A'
                ],
                index = potentialDataFrameColumns
            ),
            ignore_index = True
        )

potentialDataFrame

,Ticker,Price,30 day Average Volume,T0 Date,T0 Volume,T1 Volume,T1 Volume Ratio
0,A,122.5,1720620,2022-07-11 00:00:00,1328281,38820,N/A
1,AA,43.62,6818079,2022-07-11 00:00:00,4984276,395682,N/A
2,AAC,10.02,188711,2022-07-11 00:00:00,240526,2679,N/A
3,AACI,10.19,70638,2022-07-11 00:00:00,316,0,N/A
4,AADI,13.21,246571,2022-07-11 00:00:00,65165,2335,N/A
...,...,...,...,...,...,...,...
6209,ZWS,29.95,1099779,2022-07-11 00:00:00,791177,28850,N/A
6210,ZY,1.7,1773243,2022-07-11 00:00:00,869640,138947,N/A
6211,ZYME,6.1,1090905,2022-07-11 00:00:00,603027,77284,N/A
6212,ZYNE,1.2,376927,2022-07-11 00:00:00,240018,20119,N/A


## Data Cleaning

Clean out data with None from API

In [8]:
potentialDataFrame[potentialDataFrame.isnull().any(axis = 1)]
# potentialDataFrame[potentialDataFrame['Price'].isnull()]

,Ticker,Price,30 day Average Volume,T0 Date,T0 Volume,T1 Volume,T1 Volume Ratio
742,BIAF,None,0,NaN,NaN,0,N/A
992,CANB,NaN,NaN,2022-07-11 00:00:00,6918,NaN,N/A
2939,INTS,None,0,NaN,NaN,0,N/A
3425,LUCY,None,0,NaN,NaN,0,N/A
3773,MULG,NaN,NaN,2019-10-29,201,NaN,N/A
3844,NCPL,NaN,NaN,2022-07-08 00:00:00,4609,NaN,N/A
3846,NCRA,NaN,NaN,2022-07-11 00:00:00,477,NaN,N/A
3866,NEOV,NaN,NaN,2022-07-11 00:00:00,21200,NaN,N/A
4095,NYXH,31,0,NaN,NaN,3693,N/A
4176,ONFO,None,0,NaN,NaN,0,N/A


In [9]:
potentialDataFrame = potentialDataFrame.dropna()
potentialDataFrame

,Ticker,Price,30 day Average Volume,T0 Date,T0 Volume,T1 Volume,T1 Volume Ratio
0,A,122.5,1720620,2022-07-11 00:00:00,1328281,38820,N/A
1,AA,43.62,6818079,2022-07-11 00:00:00,4984276,395682,N/A
2,AAC,10.02,188711,2022-07-11 00:00:00,240526,2679,N/A
3,AACI,10.19,70638,2022-07-11 00:00:00,316,0,N/A
4,AADI,13.21,246571,2022-07-11 00:00:00,65165,2335,N/A
...,...,...,...,...,...,...,...
6209,ZWS,29.95,1099779,2022-07-11 00:00:00,791177,28850,N/A
6210,ZY,1.7,1773243,2022-07-11 00:00:00,869640,138947,N/A
6211,ZYME,6.1,1090905,2022-07-11 00:00:00,603027,77284,N/A
6212,ZYNE,1.2,376927,2022-07-11 00:00:00,240018,20119,N/A


Drop data with 0 Volume

In [10]:
potentialDataFrame[
    (potentialDataFrame['T0 Volume'] == 0) |
    (potentialDataFrame['T1 Volume'] == 0)
]

,Ticker,Price,30 day Average Volume,T0 Date,T0 Volume,T1 Volume,T1 Volume Ratio
3,AACI,10.19,70638,2022-07-11 00:00:00,316,0,N/A
7,AAMC,10.86,8654,2022-07-11 00:00:00,837,0,N/A
14,AAQC,10.1018,14805,2022-07-11 00:00:00,2372,0,N/A
25,ABGI,10.0,47384,2022-07-08 00:00:00,54,0,N/A
42,ACAH,9.99,29389,2022-07-11 00:00:00,594,0,N/A
...,...,...,...,...,...,...,...
6138,XPAX,9.94,17922,2022-07-08 00:00:00,628,0,N/A
6139,XPDB,10.31,148648,2022-07-11 00:00:00,126,0,N/A
6165,YOTA,9.94,246593,2022-07-11 00:00:00,7138,0,N/A
6200,ZT,9.9,100228,2022-07-11 00:00:00,20183,0,N/A


In [11]:
potentialDataFrame.drop(
    potentialDataFrame[
        (potentialDataFrame['T0 Volume'] == 0) |
        (potentialDataFrame['T1 Volume'] == 0)
    ].index,
    inplace = True
)
potentialDataFrame

/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Ticker,Price,30 day Average Volume,T0 Date,T0 Volume,T1 Volume,T1 Volume Ratio
0,A,122.5,1720620,2022-07-11 00:00:00,1328281,38820,N/A
1,AA,43.62,6818079,2022-07-11 00:00:00,4984276,395682,N/A
2,AAC,10.02,188711,2022-07-11 00:00:00,240526,2679,N/A
4,AADI,13.21,246571,2022-07-11 00:00:00,65165,2335,N/A
5,AAIC,3.22,72107,2022-07-11 00:00:00,66022,11,N/A
...,...,...,...,...,...,...,...
6209,ZWS,29.95,1099779,2022-07-11 00:00:00,791177,28850,N/A
6210,ZY,1.7,1773243,2022-07-11 00:00:00,869640,138947,N/A
6211,ZYME,6.1,1090905,2022-07-11 00:00:00,603027,77284,N/A
6212,ZYNE,1.2,376927,2022-07-11 00:00:00,240018,20119,N/A


## Calculation

Calculate Volume Ratio

In [12]:
for row in potentialDataFrame.index:
    potentialDataFrame.loc[row, 'T1 Volume Ratio'] = potentialDataFrame.loc[row, 'T1 Volume'] / potentialDataFrame.loc[row, 'T0 Volume']
    
potentialDataFrame

/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,Ticker,Price,30 day Average Volume,T0 Date,T0 Volume,T1 Volume,T1 Volume Ratio
0,A,122.5,1720620,2022-07-11 00:00:00,1328281,38820,0.029226
1,AA,43.62,6818079,2022-07-11 00:00:00,4984276,395682,0.079386
2,AAC,10.02,188711,2022-07-11 00:00:00,240526,2679,0.011138
4,AADI,13.21,246571,2022-07-11 00:00:00,65165,2335,0.035832
5,AAIC,3.22,72107,2022-07-11 00:00:00,66022,11,0.000167
...,...,...,...,...,...,...,...
6209,ZWS,29.95,1099779,2022-07-11 00:00:00,791177,28850,0.036465
6210,ZY,1.7,1773243,2022-07-11 00:00:00,869640,138947,0.159775
6211,ZYME,6.1,1090905,2022-07-11 00:00:00,603027,77284,0.12816
6212,ZYNE,1.2,376927,2022-07-11 00:00:00,240018,20119,0.083823


## Screening

Screening criteria are as follow:

- Average Volume can't be too low
- T1 Volume can't be too low
- T1 Volume is more than 5 times of T0

In [13]:
volumeLowerBound = float(5e6)
t1traget = float(5)

potentialDataFrame = potentialDataFrame[
    (volumeLowerBound < potentialDataFrame['30 day Average Volume']) &
    (volumeLowerBound < potentialDataFrame['T1 Volume']) &
    (t1traget < potentialDataFrame['T1 Volume Ratio'])
]
potentialDataFrame.sort_values(
    'Ticker',
    ascending = True,
    inplace = True
)
potentialDataFrame.reset_index(inplace = True, drop = True)

potentialDataFrame

<ipython-input-13-230acc7d45ec>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  potentialDataFrame.sort_values(


,Ticker,Price,30 day Average Volume,T0 Date,T0 Volume,T1 Volume,T1 Volume Ratio


In [14]:
','.join(potentialDataFrame['Ticker'].to_list())

''

## Data Export

For the ease of the next programe, we will also export the file in CSV version.

In [15]:
from datetime import date

today = date.today().strftime("%Y%m%d")
fileName = f'../export/yau4muk6man4_strategy_{today}_volume_rise'
csvFileName = f'{fileName}.csv'
xlsxFileName = f'{fileName}.xlsx'
sheetName = f'{today} Volume Rise'

Export to csv

In [16]:
potentialDataFrame.to_csv(csvFileName, index = False)

Export to xlsx

In [17]:
writer = pd.ExcelWriter(xlsxFileName, engine = 'xlsxwriter')
potentialDataFrame.to_excel(writer, sheet_name = sheetName, index = False)

In [18]:
backgroundColor = '#0a0a23'
fontColor = '#ffffff'

stringTemplate = writer.book.add_format(
        {
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

dollarTemplate = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

integerTemplate = writer.book.add_format(
        {
            'num_format':'#,###',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

floatTemplate = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

percentTemplate = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

In [19]:
columnFormats = {
    'A': ['Ticker', stringTemplate],
    'B': ['Price', dollarTemplate],
    'C': ['30 day Average Volume', integerTemplate],
    'D': ['T0 Date', stringTemplate],
    'E': ['T0 Volume', integerTemplate],
    'F': ['T1 Volume', integerTemplate],
    'G': ['T1 Volume Ratio', percentTemplate],
}

for column in columnFormats.keys():
    writer.sheets[sheetName].set_column(
        f'{column}:{column}',
        max(len(columnFormats[column][0]), 10),
        columnFormats[column][1]
    )
    writer.sheets[sheetName].write(
        f'{column}1',
        columnFormats[column][0],
        columnFormats[column][1]
    )

In [20]:
writer.save()